# Run a single test

In [1]:
import os, pprint

In [2]:
EXAMPLES_FOLDER = "../examples"

In [3]:
[f for f in os.listdir(EXAMPLES_FOLDER) if f not in [".DS_Store", ".ipynb_checkpoints"]]

['.tools',
 'change_case',
 'multivariate',
 'results',
 'sacurine',
 'workflow-test-suite-full.yml',
 'workflow-test-suite-min.yml',
 'workflows.json']

In [6]:
suite_conf_filename = os.path.join(EXAMPLES_FOLDER, "workflow-test-suite-min.yml")

In [7]:
import ruamel.yaml as yaml
with open(suite_conf_filename, "r") as fp:
    data = yaml.load(fp)
    print yaml.round_trip_dump(data, indent=2, block_seq_indent=3)

enable_logger: false
workflows:
  change_case:
    expected:
      OutputText: change_case/expected_output
    inputs:
      InputText: change_case/input
    file: change_case/workflow.ga
  multivariate:
    expected:
      variableMetadata_out: multivariate/variableMetadata_out
      sampleMetadata_out: multivariate/sampleMetadata_out
    inputs:
      DataMatrix: multivariate/dataMatrix.tsv
      SampleMetadata: multivariate/sampleMetadata.tsv
      VariableMetadata: multivariate/variableMetadata.tsv
    params:
      3:
        predI: '1'
        respC: gender
        orthoI: NA
        testL: 'FALSE'
    file: multivariate/workflow.ga



In [8]:
from wft4galaxy import WorkflowTestConfiguration

In [9]:
help(WorkflowTestConfiguration.load)

Help on function load in module wft4galaxy:

load(filename='workflows.yml', workflow_test_name=None, file_format='YAML', output_folder=None)
    Load the configuration of a workflow test suite or a single workflow test
    from a YAML or JSON configuration file.
    
    :type filename: str
    :param filename: the path of the file containing the suite definition
    
    :type workflow_test_name: str
    :param workflow_test_name: the optional name of a workflow test
    
    :type file_format: str
    :param file_format: ``YAML`` or ``JSON``
    
    :rtype: dict or :class:`WorkflowTestConfiguration`
    :return: a dictionary containing the configuration of a workflow test suite or
        a single workflow test configuration (i.e., an instance of :class:`WorkflowTestConfiguration`)



In [10]:
test_conf = WorkflowTestConfiguration.load(suite_conf_filename, "change_case")

In [11]:
print test_conf

WorkflowTestConfig: name=change_case, file=change_case/workflow.ga, inputs=[InputText], expected_outputs=[OutputText]


In [12]:
from wft4galaxy import WorkflowTestRunner

In [13]:
help(WorkflowTestRunner.new_instance)

Help on function new_instance in module wft4galaxy:

new_instance(workflow_test_config, galaxy_url=None, galaxy_api_key=None)
    Factory method to create and initialize a new :class:`WorkflowTestRunner` instance.
    
    :type workflow_test_config: :class:`WorkflowTestConfiguration`
    :param workflow_test_config: the configuration of a workflow test
    
    :type galaxy_url: str
    :param galaxy_url: url of your Galaxy server instance.  If ``none``, the environment variable
        ``BIOBLEND_GALAXY_URL`` is used. An error is raised when such a variable cannot be found.
    
    :type galaxy_api_key: str
    :param galaxy_api_key: an API key from your Galaxy server instance.  If ``none``, the environment variable
        ``BIOBLEND_GALAXY_API_KEY`` is used. An error is raised when such a variable cannot be found.
    
    :rtype: :class:`WorkflowTestRunner`
    :return: a :class:`WorkflowTestRunner` instance



In [14]:
test_runner = WorkflowTestRunner.new_instance(test_conf)

In [15]:
help(test_runner.run_test)

Help on method run_test in module wft4galaxy:

run_test(self, base_path=None, inputs=None, params=None, expected_outputs=None, output_folder=None, disable_assertions=None, disable_cleanup=None, enable_logger=None, enable_debug=None) method of wft4galaxy.WorkflowTestRunner instance
    Run the workflow test which this runner is associated to.
    The parameters ``base_path``, ``inputs``, ``outputs``, ``expected_outputs``
    ``output_folder``, ``disable_assertions``, ``disable_cleanup``, ``enable_logger``, ``enable_debug``
    can be provided to override the corresponding defined in the :class:`WorkflowTestConfiguration` instance
    which this runner is related to (see :class:`WorkflowTestConfiguration` for more details).
    
    :rtype: :class:`WorkflowTestResult`
    :return: the :class:`WorkflowTestResult` instance which represents the test result



In [16]:
test_result = test_runner.run_test(enable_debug=True)

2016-11-09 07:20:21,961 DEBUG: Using default params
2016-11-09 07:20:21,985 DEBUG: Checking required tools ...
2016-11-09 07:20:22,042 DEBUG: Missing tools: None
2016-11-09 07:20:22,043 DEBUG: Checking required tools: DONE
2016-11-09 07:20:22,500 INFO: Create a history '_WorkflowTestHistory_9801da68-a644-11e6-8d32-a8667f33b7fc' (id: u'0d16186aaff7cbfd')
2016-11-09 07:20:24,401 INFO: Workflow '_WorkflowTest_Change Case (imported from API)' (id: 0d16186aaff7cbfd) running ...
2016-11-09 07:20:37,107 INFO: Workflow '_WorkflowTest_Change Case (imported from API)' (id: 0d16186aaff7cbfd) executed
2016-11-09 07:20:37,109 INFO: Checking test output: ...
2016-11-09 07:20:37,110 DEBUG: Checking OUTPUT 'OutputText' ...
2016-11-09 07:20:37,295 DEBUG: Downloaded output OutputText: dataset_id '38d70b27d179c236', filename 'results/change_case/OutputText'
2016-11-09 07:20:37,296 DEBUG: Configured comparator function: None
2016-11-09 07:20:37,296 DEBUG: Using default comparator....
2016-11-09 07:20:37,3

In [17]:
print("Test %s:\n\t - workflow: [%s] \n\t - results: %r" % \
      (test_result.test_id, test_result.workflow.name, test_result.results))

Test 9801da68-a644-11e6-8d32-a8667f33b7fc:
	 - workflow: [_WorkflowTest_Change Case (imported from API)] 
	 - results: {u'OutputText': True}


In [18]:
print "Outputs: ", test_result.results.keys()

Outputs:  [u'OutputText']


In [19]:
test_result.passed(), test_result.failed()

(True, False)

In [22]:
test_result.check_output("OutputText")

True